In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive")

Mounted at /content/MyDrive


In [ ]:
# /relation extraction code

import os
import regex as re

# Concatenate all the ann files in the "train2" folder
path = '/content/MyDrive/MyDrive/data/scienceie2017_train/train2'
# /content/MyDrive/MyDrive/data/scienceie2017_train/train2

all_lines = []

for filename in os.listdir(path):
    if filename.endswith('.ann'):
        with open(os.path.join(path, filename), 'r') as f:
            lines = f.readlines()
            id_to_text = {}
            for ind, line in enumerate(lines):
              if line.startswith('T'):
                line_parts = re.sub(r'\s+', '\t', line.strip())
                line_parts = re.split(r'\t', line_parts)
                key = line_parts[0]
                text = line_parts[4:]
                text = ' '.join(map(str, text))
                id_to_text[key] = text
            for ind, line in enumerate(lines):
              if line.startswith('R') or line.startswith('*'):
                line = line.replace('Arg1:', '').replace('Arg2:', '')
                line = re.sub(r'\s+', '\t', line.strip())
                line = re.split(r'\t', line)
                arg1_id = line[-2]
                arg2_id = line[-1]
                try:
                  arg1_text = id_to_text[arg1_id]
                except:
                  print('eror hi eroor 1', arg1_id, id_to_text)
                line[-2] = arg1_text + " &"
                try:
                  arg2_text = id_to_text[arg2_id]
                except:
                  print('eror hi eroor 2')
                line[-1] = arg2_text
                line = ' '.join(map(str, line))
                lines[ind] = line
            all_lines.extend(lines)

# extracting relation tuples
relations = []
for line in all_lines:
    if line.startswith('R') or line.startswith('*'):
      relations.append(line)

for i in range(10):
  print(relations[i])
# # Extract relation tuples
# 
# for ind, line in enumerate(all_lines):
#     if line.startswith('R'):
#       all_lines[ind] = all_lines[ind].replace('Arg1:', '').replace('Arg2:', '')

# id_to_text = {}
# for line in all_lines:
#     if line.startswith('T'):
#         line_parts = line.split('\t')
#         key = line_parts[0]
#         text = line_parts[-1].strip()
#         id_to_text[key] = text

# for ind, line in enumerate(all_lines):
#     if line.startswith('R') or line.startswith('*'):
#         print(line)

# for line in all_lines:
#     if line.startswith('R') or line.startswith('*'):
#       print(line)
#       break



R1 Hyponym-of MINERAL (MINeral ERror AnaLysis) & a new MATLAB® based program
* Synonym-of La0.6Sr0.4CoO3−δ & LSC
* Synonym-of yttria stabilized zirconia & YSZ
* Synonym-of T3 PSR & SA
* Synonym-of curvature-dependent polygonization & increasing/decreasing the size of triangles according to the local curvature
R1 Hyponym-of relative permeability & difference in magnetic properties
R2 Hyponym-of ferrite & ferromagnetic
R3 Hyponym-of austenite & paramagnetic
R4 Hyponym-of decarburisation & variation in ferrite content with depth
* Synonym-of finite element & FE


In [ ]:
train_data = []
new_relations = []
for ind, relation in enumerate(relations):
    # Split the relation into its components
    components = relation.split(' ')
    new_relations.append(components[1:])
    # Extract the label (Hyponym-of, Synonym-of, etc.)
#     relation

#     # Extract the two phrases and create a tuple
#     for i in range(1, len(components)):
#         phrase1 = components[i].split(' & ')[0]
#         phrase2 = components[0].split(' ')[-1]
#         train_data.append((phrase1, phrase2, label))
labels = []
text1 = []
text2 = []
for i, relation in enumerate(new_relations):
  # print(new_relations[i])
  labels.append(new_relations[i][0])
  # print(new_relations[i][0])
  newindex = new_relations[i].index("&")
  # print("text1 = ", ' '.join(new_relations[i][1:newindex]))
  # print("text2 = ", new_relations[i][newindex+1:])
  text1.append((' '.join(new_relations[i][1:newindex])))
  text2.append((' '.join(new_relations[i][newindex+1:])))
# for i in range(10):
#   print(labels[i])
#   print(text1[i])
#   print(text2[i])
print(len(labels))
print(len(text1))
print(len(text2))

673
673
673


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.9 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import tqdm

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)
label2id = {"Hyponym-of": 0, "Synonym-of": 1}
labels = [label2id[label] for label in labels]
# Tokenize the texts
tokenized_texts = tokenizer(text1, text2, padding=True, truncation=True, return_tensors='pt').to(device)
print(len(labels))
print(len(text1))
print(len(text2))
# Create a TensorDataset
print(tokenized_texts['input_ids'].shape)
print(tokenized_texts['attention_mask'].shape)
print(torch.tensor(labels).shape)
dataset = TensorDataset(tokenized_texts['input_ids'], tokenized_texts['attention_mask'], torch.tensor(labels).to(device))

# Define the dataloader
dataloader = DataLoader(dataset, batch_size=2)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
model.train()
for epoch in range(5):
    print(f"Epoch {epoch+1}")
    progress_bar = tqdm(dataloader)
    for input_ids, attention_mask, label in progress_bar:
        input_ids, attention_mask, label = input_ids.to(device), attention_mask.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = criterion(outputs.logits, label)
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'batch_loss': loss.item()})
    
# Save the model
model_path = 'bert_relation_extraction.pt'
torch.save(model.state_dict(), model_path)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

673
673
673
torch.Size([673, 45])
torch.Size([673, 45])
torch.Size([673])
Epoch 1


  0%|          | 0/337 [00:00<?, ?it/s]

Epoch 2


  0%|          | 0/337 [00:00<?, ?it/s]

...

  0%|          | 0/337 [00:00<?, ?it/s]

Epoch 4


  0%|          | 0/337 [00:00<?, ?it/s]

Epoch 5


  0%|          | 0/337 [00:00<?, ?it/s]

In [ ]:
path = '/content/MyDrive/MyDrive/data/scienceie2017_dev/dev'
# /content/MyDrive/MyDrive/data/scienceie2017_train/train2

all_lines = []

for filename in os.listdir(path):
    if filename.endswith('.ann'):
        with open(os.path.join(path, filename), 'r') as f:
            lines = f.readlines()
            id_to_text = {}
            for ind, line in enumerate(lines):
              if line.startswith('T'):
                line_parts = re.sub(r'\s+', '\t', line.strip())
                line_parts = re.split(r'\t', line_parts)
                key = line_parts[0]
                text = line_parts[4:]
                text = ' '.join(map(str, text))
                id_to_text[key] = text
            for ind, line in enumerate(lines):
              if line.startswith('R') or line.startswith('*'):
                line = line.replace('Arg1:', '').replace('Arg2:', '')
                line = re.sub(r'\s+', '\t', line.strip())
                line = re.split(r'\t', line)
                arg1_id = line[-2]
                arg2_id = line[-1]
                try:
                  arg1_text = id_to_text[arg1_id]
                except:
                  print('eror hi eroor 1', arg1_id, id_to_text)
                line[-2] = arg1_text + " &"
                try:
                  arg2_text = id_to_text[arg2_id]
                except:
                  print('eror hi eroor 2')
                line[-1] = arg2_text
                line = ' '.join(map(str, line))
                lines[ind] = line
            all_lines.extend(lines)

# extracting relation tuples
relations = []
for line in all_lines:
    if line.startswith('R') or line.startswith('*'):
      relations.append(line)

train_data = []
new_relations = []
for ind, relation in enumerate(relations):
    # Split the relation into its components
    components = relation.split(' ')
    new_relations.append(components[1:])
    # Extract the label (Hyponym-of, Synonym-of, etc.)
#     relation

#     # Extract the two phrases and create a tuple
#     for i in range(1, len(components)):
#         phrase1 = components[i].split(' & ')[0]
#         phrase2 = components[0].split(' ')[-1]
#         train_data.append((phrase1, phrase2, label))
test_labels = []
test_text1 = []
test_text2 = []
for i, relation in enumerate(new_relations):
  # print(new_relations[i])
  test_labels.append(new_relations[i][0])
  # print(new_relations[i][0])
  newindex = new_relations[i].index("&")
  # print("text1 = ", ' '.join(new_relations[i][1:newindex]))
  # print("text2 = ", new_relations[i][newindex+1:])
  test_text1.append((' '.join(new_relations[i][1:newindex])))
  test_text2.append((' '.join(new_relations[i][newindex+1:])))
# for i in range(10):
#   print(labels[i])
#   print(text1[i])
#   print(text2[i])

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load the trained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.load_state_dict(torch.load('bert_relation_extraction.pt'))

# Set the device to cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Tokenize the test data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_test = tokenizer(test_text1, test_text2, padding=True, truncation=True, return_tensors='pt')

# Create a TensorDataset for the tokenized test data and labels
label2id = {"Hyponym-of": 0, "Synonym-of": 1}
test_labels = [label2id[label] for label in test_labels]
test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'], torch.tensor(test_labels))

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=2)

# Set the model to evaluation mode
model.eval()

# Initialize variables to keep track of accuracy and predictions
total = 0
correct = 0
predictions = []

# Iterate over the test DataLoader and get the model predictions
for input_ids, attention_mask, label in test_dataloader:
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    label = label.to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, preds = torch.max(logits, dim=1)
    
    # Add the predictions to the list of all predictions
    predictions.extend(preds.tolist())
    
    # Update the accuracy
    total += label.size(0)
    correct += (preds == label).sum().item()

# Compute the accuracy
accuracy = 100 * correct / total
print(f'Test accuracy: {accuracy:.2f}%')
# print(predictions)
# Compute the F1 score
from sklearn.metrics import f1_score

f1 = f1_score(test_labels, predictions, average='micro')

print(f'Test F1 score: {f1:.2f}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Test accuracy: 80.36%
Test F1 score: 0.80


In [ ]:
# classification


import os
import regex as re

# Concatenate all the ann files in the "train2" folder
path = '/content/MyDrive/MyDrive/data/scienceie2017_train/train2'
# /content/MyDrive/MyDrive/data/scienceie2017_train/train2
all_lines = []
for filename in os.listdir(path):
    if filename.endswith('.ann'):
        with open(os.path.join(path, filename), 'r') as f:
            lines = f.readlines()
            all_lines.extend(lines)

# Extract the class and text from each line that starts with "T"
selected_lines = []
d = {'material':0, 'task':1, 'process':2}
for line in all_lines:
    if line.startswith('T'):
        try:
          line = re.sub(r'\s+', '\t', line.strip())
          x = line.split('\t')
          id_, class_, start, end, text = x[0], x[1], x[2], x[3], x[4:]
          class_ = class_.lower()
          text = ' '.join(map(str, text)).lower()
          class_ = class_.split()[0]  # Extract the first part of the class, e.g. "Process" from "Process 317 349"
          selected_lines.append((d[class_], text))
          
        except:
          print("boyyah")

# Print the result
print(selected_lines[10])


(0, 'probe amph')


In [ ]:
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

Device available now: cuda


In [ ]:
import pandas as pd
df = pd.DataFrame(selected_lines, columns=['class', 'text'])
df
train_df = df
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(df):
    input_ids = []
    attention_masks = []
    labels = []

    for index, row in df.iterrows():
        input_encoding = tokenizer.encode_plus(
            row['text'],
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids.append(input_encoding['input_ids'])
        attention_masks.append(input_encoding['attention_mask'])
        labels.append(row['class'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

train_input_ids, train_attention_masks, train_labels = preprocess_data(train_df)
# test_input_ids, test_attention_masks, test_labels = preprocess_data(test_df)





In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

for epoch in range(5):
    running_loss = 0.0
    for ind, batch in enumerate(train_loader):
        model.train()
        batch_input_ids, batch_attention_masks, batch_labels = batch
        batch_input_ids = batch_input_ids.to(device)
        batch_attention_masks = batch_attention_masks.to(device)
        batch_labels = batch_labels.to(device)
        print(ind/len(train_loader))

        optimizer.zero_grad()

        outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} loss: {running_loss/len(train_loader)}")

0.0
0.004739336492890996
0.009478672985781991
0.014218009478672985
0.018957345971563982
0.023696682464454975
0.02843601895734597
0.03317535545023697
0.037914691943127965
0.04265402843601896
0.04739336492890995
0.052132701421800945
0.05687203791469194
0.061611374407582936
0.06635071090047394
0.07109004739336493
0.07582938388625593
0.08056872037914692
0.08530805687203792
0.09004739336492891
0.0947867298578199
0.0995260663507109
0.10426540284360189
0.10900473933649289
0.11374407582938388
0.11848341232227488
0.12322274881516587
0.12796208530805686
0.13270142180094788
0.13744075829383887
0.14218009478672985
0.14691943127962084
0.15165876777251186
0.15639810426540285
0.16113744075829384
0.16587677725118483
0.17061611374407584
0.17535545023696683
0.18009478672985782
0.1848341232227488
0.1895734597156398
0.1943127962085308
0.1990521327014218
0.2037914691943128
0.20853080568720378
0.2132701421800948
0.21800947867298578
0.22274881516587677
0.22748815165876776
0.23222748815165878
0.23696682464454

In [ ]:
model_path = '/content/model__classification'
model.save_pretrained(model_path)

model = BertForSequenceClassification.from_pretrained(model_path).to(device)


In [ ]:
import os
import regex as re
from sklearn.metrics import f1_score, precision_score, recall_score

# Concatenate all the ann files in the "train2" folder
path = '/content/MyDrive/MyDrive/data/scienceie2017_dev/dev'
# /content/MyDrive/MyDrive/data/scienceie2017_train/train2
all_lines = []
for filename in os.listdir(path):
    if filename.endswith('.ann'):
        with open(os.path.join(path, filename), 'r') as f:
            lines = f.readlines()
            all_lines.extend(lines)

# Extract the class and text from each line that starts with "T"
selected_lines = []
d = {'material':0, 'task':1, 'process':2}
for line in all_lines:
    if line.startswith('T'):
        try:
          line = re.sub(r'\s+', '\t', line.strip())
          x = line.split('\t')
          id_, class_, start, end, text = x[0], x[1], x[2], x[3], x[4:]
          class_ = class_.lower()
          text = ' '.join(map(str, text)).lower()
          class_ = class_.split()[0]  # Extract the first part of the class, e.g. "Process" from "Process 317 349"
          selected_lines.append((d[class_], text))
          
        except:
          print("boyyah")

# Print the result
print(selected_lines[10])

test_df = pd.DataFrame(selected_lines, columns=['class', 'text'])
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(df):
    input_ids = []
    attention_masks = []
    labels = []

    for index, row in df.iterrows():
        input_encoding = tokenizer.encode_plus(
            row['text'],
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids.append(input_encoding['input_ids'])
        attention_masks.append(input_encoding['attention_mask'])
        labels.append(row['class'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

# train_input_ids, train_attention_masks, train_labels = preprocess_data(train_df)
test_input_ids, test_attention_masks, test_labels = preprocess_data(test_df)

model.eval()

# Move the test data to the device
test_input_ids = test_input_ids.to(device)
test_attention_masks = test_attention_masks.to(device)
test_labels = test_labels.to(device)


# Disable gradient calculations to save memory and time
with torch.no_grad():
    # Forward pass to get the model's predictions
    logits = model(test_input_ids, attention_mask=test_attention_masks)[0].to(device)
    probabilities = torch.softmax(logits, dim=1)
    # Get the predicted class (the class with highest probability)
    predicted_class = torch.argmax(probabilities, dim=1)


# Calculate the accuracy, F1 score, precision, and recall of the model on the test set
correct_predictions = torch.sum(predicted_class == test_labels)
accuracy = correct_predictions.item() / len(test_labels)
f1 = f1_score(test_labels.cpu().numpy(), predicted_class.cpu().numpy(), average='macro')
precision = precision_score(test_labels.cpu().numpy(), predicted_class.cpu().numpy(), average='macro')
recall = recall_score(test_labels.cpu().numpy(), predicted_class.cpu().numpy(), average='macro')

print('Accuracy on test set: {:.2f}%'.format(accuracy * 100))
print('F1 score on test set: {:.2f}'.format(f1))
print('Precision on test set: {:.2f}'.format(precision))
print('Recall on test set: {:.2f}'.format(recall))

(0, 'capillary waves')
Accuracy on test set: 79.26%
F1 score on test set: 0.74
Precision on test set: 0.73
Recall on test set: 0.74


In [ ]:
import os
import regex as re

# Concatenate all the ann files in the "train2" folder
path = '/content/MyDrive/MyDrive/data/scienceie2017_train/train2'
# /content/MyDrive/MyDrive/data/scienceie2017_train/train2


all_lines = []
for filename in os.listdir(path):
    if filename.endswith('.ann'):
        with open(os.path.join(path, filename), 'r') as f:
            lines = f.readlines()
            for ind, line in enumerate(lines):
              if line.startswith('R'):
                lines[ind] = lines[ind].replace('Arg1:', '').replace('Arg2:', '')
            all_lines.extend(lines)

# Extract relation tuples
relations = []
for ind, line in enumerate(all_lines):
    if line.startswith('R'):
      all_lines[ind] = all_lines[ind].replace('Arg1:', '').replace('Arg2:', '')

id_to_text = {}
for line in all_lines:
    if line.startswith('T'):
        line_parts = line.split('\t')
        key = line_parts[0]
        text = line_parts[-1].strip()
        id_to_text[key] = text

for ind, line in enumerate(all_lines):
    if line.startswith('R') or line.startswith('*'):
        print(line)

for line in all_lines:
    if line.startswith('R') or line.startswith('*'):
      print(line)
      break



R1	Hyponym-of T1 T2	

*	Synonym-of T12 T13

*	Synonym-of T17 T18

*	Synonym-of T3 T4 T12

*	Synonym-of T11 T13

R1	Hyponym-of T3 T2	

R2	Hyponym-of T9 T10	

R3	Hyponym-of T13 T14	

R4	Hyponym-of T16 T17	

*	Synonym-of T22 T23

R1	Hyponym-of T5 T6	

R2	Hyponym-of T4 T6	

R3	Hyponym-of T3 T6	

R4	Hyponym-of T2 T6	

R1	Hyponym-of T5 T6	

*	Synonym-of T7 T8

R1	Hyponym-of T22 T21	

R1	Hyponym-of T14 T13	

*	Synonym-of T16 T17

*	Synonym-of T9 T18

*	Synonym-of T24 T25

*	Synonym-of T3 T4

*	Synonym-of T5 T6

R2	Hyponym-of T13 T12	

R1	Hyponym-of T3 T4	

R2	Hyponym-of T7 T8	

R3	Hyponym-of T13 T12	

R4	Hyponym-of T23 T12	

R5	Hyponym-of T24 T12	

R6	Hyponym-of T25 T12	

R7	Hyponym-of T26 T12	

R8	Hyponym-of T27 T12	

*	Synonym-of T7 T6

R1	Hyponym-of T6 T7	

R1	Hyponym-of T1 T2	

R2	Hyponym-of T1 T4	

R3	Hyponym-of T6 T7	

R4	Hyponym-of T8 T9	

*	Synonym-of T3 T2

R1	Hyponym-of T2 T6	

R2	Hyponym-of T4 T6	

*	Synonym-of T2 T3 T5

R1	Hyponym-of T2 T4	

R2	Hyponym-of T16 T17	

R3	Hyponym-of T

In [ ]:
!pip install sentence-transformers
!pip install icecream


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=ffb2db520219c11aefb66037fc5dd300d75d98ee486b93721633b876d026d47a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
from transformers import AutoTokenizer


def parseKeywords(fileName):
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
    
    all_kws = []
    with open(fileName, 'r') as f:
        lines = f.readlines()

        for line in lines:
            words = line.split('\t')
            if words[0][0] != 'R':
                kw_tag = re.sub('\n', '', words[-1])
                kws = re.split('[-.:\/\' ]', kw_tag.lower())

            all_kws.append(kws)
    return all_kws


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from sentence_transformers import SentenceTransformer

from icecream import ic
from tqdm import tqdm
import os

def get_file_names(dir_path):
    file_names = [file.split('.')[0] for file in os.listdir(dir_path)]
    file_names = list(set(file_names))

    pairs = []
    for name in file_names:
        pairs.append([os.path.join(dir_path, name + ".txt"), os.path.join(dir_path, name + ".ann")])
    
    return pairs

def get_unsupervised_keywords(doc, model, n_gram_range=(1,1), top_n=10):
    # extract all possible keyphrase candidates (n_grams)
    stop_words="english"
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
    candidates = count.get_feature_names_out()

    # convert candidates to embeddings
    doc_emb = model.encode([doc])
    candidate_embs = model.encode(candidates)

    # get the closest embeddings (lowest dist -> most closely related embeddings)
    distances = cosine_distances(doc_emb, candidate_embs)
    keywords = [candidates[index].split(' ') for index in distances.argsort()[0][:top_n]]

    distances.sort()
    dists = distances[0, :top_n]

    # for i, j in zip(keywords, dists):
    #     print(i,j)

    return keywords, candidates



"""
istilbert — base-nli-stsb-mean-tokens performs well for semantic similarity
xlm-r-distilroberta-base-paraphase-v1 performs well for paraphrase identification

all-MiniLM-L12-v2 -- modern, recommended version of the above

all-mpnet-base-v2 -- best 
"""
model = SentenceTransformer('all-MiniLM-L12-v2')
# model = SentenceTransformer('all-mpnet-base-v2')
# file_pairs = get_file_names("/home/aneesh/UbuntuStorage/Homework/INLP/SciTraction/SemEvalData/scienceie2017_dev/dev")   # [text, annotation]
# file_pairs = get_file_names("./data/scienceie2017_dev/dev")   # [text, annotation]
file_pairs = get_file_names("/content/MyDrive/MyDrive/data/scienceie2017_train/train2")   # [text, annotation]

# doc = "Poor oxidation behavior is the major barrier to the increased use of Ti-based alloys in high-temperature structural applications. The demand to increase the service temperature of these alloys beyond 550°C (the typical temperature limit) requires careful study to understand the role that composition has on the oxidation behavior of Ti-based alloys [1–3]. The attempt to overcome this limitation in Ti-based alloys has led to the production of alloys with substantially improved oxidation resistance such as β-21S and also development of coatings and pre-oxidation techniques [1,4–6]. While it is tempting to extrapolate the oxidation behavior (e.g. oxidation rate law, depth of oxygen ingress and scale thickness) observed for a limited number of compositions under a certain oxidation condition to a broader compositional range, there are numerous examples in the literature where deviations from the expected relations are observed [7,8]."
# doc = "The Age of Enlightenment or the Enlightenment,[note 2] also known as the Age of Reason, was an intellectual and philosophical movement that occurred in Europe in the 17th and 18th centuries, with global influences and effects.[2][3] The Enlightenment included a range of ideas centered on the value of human happiness, the pursuit of knowledge obtained by means of reason and the evidence of the senses, and ideals such as natural law, liberty, progress, toleration, fraternity, constitutional government, and separation of church and state.[4][5] The Enlightenment was preceded by the Scientific Revolution and the work of Francis Bacon, John Locke, among others. Some date the beginning of the Enlightenment to the publication of René Descartes' Discourse on the Method in 1637, featuring his famous dictum, Cogito, ergo sum (\"I think, therefore I am\"). Others cite the publication of Isaac Newton's Principia Mathematica (1687) as the culmination of the Scientific Revolution and the beginning of the Enlightenment. European historians traditionally date its beginning with the death of Louis XIV of France in 1715 and its end with the 1789 outbreak of the French Revolution. Many historians now date the end of the Enlightenment as the start of the 19th century, with the latest proposed year being the death of Immanuel Kant in 1804. Philosophers and scientists of the period widely circulated their ideas through meetings at scientific academies, Masonic lodges, literary salons, coffeehouses and in printed books, journals,[7] and pamphlets. The ideas of the Enlightenment undermined the authority of the monarchy and the Catholic Church and paved the way for the political revolutions of the 18th and 19th centuries. A variety of 19th century movements including liberalism, communism, and neoclassicism trace their intellectual heritage to the Enlightenment. The central doctrines of the Enlightenment were individual liberty and religious tolerance, in opposition to an absolute monarchy and the fixed dogmas of the Church. The concepts of utility and sociability were also crucial in the dissemination of information that would better society as a whole. The Enlightenment was marked by an increasing awareness of the relationship between the mind and the everyday media of the world,[9] and by an emphasis on the scientific method and reductionism, along with increased questioning of religious orthodoxy—an attitude captured by Kant's essay Answering the Question: What is Enlightenment, where the phrase Sapere aude (Dare to know) can be found.[10] "

# init recall and precision values for keyphrases until a cutoff length
cutoff_len = 5
all_stats = {}

quiet = True

for pair in tqdm(file_pairs):
    # parse ground truth keywords and text
    with open(pair[0], 'r') as f:
        doc = f.readline()

    gt_kws = parseKeywords(pair[1])
    
    # sort keywords acc to length
    kws_len_dict = {}
    for kw in gt_kws:
        if len(kw) not in kws_len_dict.keys():
            kws_len_dict[len(kw)] = [kw]
        else:
            kws_len_dict[len(kw)].append(kw)

    if not quiet:
        ic(gt_kws)
        print(kws_len_dict)
        print()

    for i in range(1, cutoff_len):
        if i in kws_len_dict.keys():
            pred_kws, candidates = get_unsupervised_keywords(doc, model, (i,i), len(kws_len_dict[i]))       # generate as many keywords based on no. of gt keywords of that length

            # check if each ground truth kw was generated
            correctly_predicted = 0
            incorrectly_predicted = 0
            missed_kws = 0
            total = len(candidates)

            if not quiet:
                print("Length: %d" % i)

            for kw in kws_len_dict[i]:

                if not quiet:
                    print("\t\t\tkw: %s" % kw)
                    print("\t\t\tkw: %s" % pred_kws)

                if kw in pred_kws:
                    correctly_predicted += 1
                else:
                    missed_kws += 1

            for pred_kw in pred_kws:
                if pred_kws not in kws_len_dict[i]:
                    incorrectly_predicted += 1

            discarded = total - len(pred_kws)

            
            prec = correctly_predicted/(correctly_predicted + incorrectly_predicted)
            reca = correctly_predicted/(correctly_predicted + missed_kws)
            
            if prec == 0 and reca == 0:
                fsco = 0
            else:
                fsco = 2 * (prec * reca)/(prec + reca)
            
            if not quiet:
                ic(correctly_predicted, incorrectly_predicted, missed_kws, discarded)
                print()
                ic(prec, reca, fsco)
                print()
                print()

            if i in all_stats.keys():
                all_stats[i]["tp"] += correctly_predicted
                all_stats[i]["tn"] += discarded
                all_stats[i]["fp"] += incorrectly_predicted
                all_stats[i]["fn"] += missed_kws

                all_stats[i]["prec"].append(prec)
                all_stats[i]["reca"].append(reca)
            else:
                all_stats[i] = {"tp": correctly_predicted, "tn": discarded, "fp": incorrectly_predicted, "fn": missed_kws, "prec": [prec], "reca": [reca]}

print("-"*100)

print("Final results:")
valid_lengths = list(all_stats.keys())
valid_lengths.sort()
for length in valid_lengths:
    print("Keyphrase length: %d" % length)

    if all_stats[length]["tp"] != 0:
        precision = all_stats[length]["tp"]/(all_stats[length]["tp"] + all_stats[length]["fp"])
        recall = all_stats[length]["tp"]/(all_stats[length]["tp"] + all_stats[length]["fn"])
    else:
        precision = 0
        recall = 0

    if precision == 0 and recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall)/(precision + recall)
        
    avg_prec = 0
    avg_reca = 0
    for k in all_stats[length]["prec"]:
        avg_prec += k
    for k in all_stats[length]["reca"]:
        avg_reca += k

    avg_prec /= len(all_stats[length]["prec"])
    avg_reca /= len(all_stats[length]["reca"])


    ic(precision, recall, f1_score, avg_prec, avg_reca)
    print()




  0%|          | 0/350 [00:00<?, ?it/s]

100%|██████████| 350/350 [18:28<00:00,  3.17s/it]
ic| precision: 0.28884615384615386
    recall: 0.40616549486208764
    f1_score: 0.33760395594515624
    avg_prec: 0.19110905054582142
    avg_reca: 0.30315973110418726
ic| precision: 0.20575221238938052
    recall

----------------------------------------------------------------------------------------------------
Final results:
Keyphrase length: 1

Keyphrase length: 2


: 0.2590529247910863
    f1_score: 0.22934648581997533
    avg_prec: 0.1517551531225871
    avg_reca: 0.21671395804285726
ic| precision: 0.1293800539083558
    recall: 0.14860681114551083
    f1_score: 0.13832853025936603
    avg_prec: 0.09298609483794666
    avg_reca: 0.13530666007517866
ic| precision: 0.046169989506820566
    recall: 0.0484048404840484
    f1_score: 0.047261009667024706
    avg_prec: 0.03086773887744762
    avg_reca: 0.04670211126521807



Keyphrase length: 3

Keyphrase length: 4



In [ ]:
# concat files
# classes = ['material', 'process', 'task']
# l = line.lower().split()
# if l[1] not in classes
# keyphrases = l[4:]
# keyphrases concat --> str

Here is the format of the ann file

T1	Process 205 238	model of the smallest thicknesses
T2	Task 197 238	build a model of the smallest thicknesses
T3	Material 268 274	sample
T4	Process 317 349	partitioning the inspection data
T5	Material 334 349	inspection data
*	Synonym-of T4 T5
T6	Material 754 761	diamond
R1 Hyponym-of Arg1:T1 Arg2:T2
R2 Hyponym-of Arg1:T3 Arg2:T6
T7	Material 798 808	2-propanol
